In [9]:
import re
# pid = !netstat -a -v | grep 8050
# pid = int(re.split('\s+', pid[0])[-4])
# !kill {pid}

import pandas as pd

import jupyter_dash
from dash import html
import dash
import dash_bootstrap_components as dbc
from dash import dcc
from dash.dependencies import Input, Output

from IPython.lib import backgroundjobs as bg
from IPython.display import IFrame as WebBrowser
import plotly.express as px

jobs = bg.BackgroundJobManager()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


width = 700
height = int(1.6180339887 * width)

style_header = {"text-align": "center"}
style_subtitle = {"font-style": "italic", "color":"grey"}

mimic_url = 'https://mimic.mit.edu/docs/iii/'
dash_url = 'https://dash.plotly.com/'

style_main_adaptive = { 'lg':{'size':8, 'offset':2}, 'md':{'size':12, 'offset':2}  } 

app_title = html.H1('Welcome to the dashboard page of HIS exam', style = style_header)
app_p = html.P('We choose to implement an html page written in python with bootstrap that permits to visualize pmdata dataset stored previously on the database with ORM. We implement 3 different types of dashboard, each with a callback function to filter data visualized:')
app_title_dash1 = html.H2('PIE GRAPH - Dashboard 1')
subtitle_dash1 = html.H4('Visualize the minutes in different activity for partecipants', style = style_subtitle)

app_title_dash2 = html.H2('BAR GRAPH - Dashboard 2')
subtitle_dash2 = html.H4('Visualize the minutes in different activity for every partecipant in the selected age range', style = style_subtitle)

app_title_dash3 = html.H2('SCATTER GRAPH - Dashboard 3')
subtitle_dash3 = html.H4('Visualize the bpm scatter over vt and the number of calories as dimension of the dots', style = style_subtitle)

app_items = html.Ul([ 
                html.Li(['Pie graph']),
                html.Li(['Bar graph']),
                html.Li(['Scatter graph']) 
            ]
        )
hr = html.Hr()


from pony.orm import Database, Required, Optional, PrimaryKey, Set, select, count, db_session
from  datetime import datetime,date
import psycopg2

#Connessione al DB !RIcordarsi di far andare il server!
DB = Database()
DB.bind(provider='postgres', 
        database='postgres',
        user='postgres', 
        port='5433',  #Linda: 5432
        host='localhost',
        password='admin'
        )

class Partecipant(DB.Entity):
    _table_  = 'partecipant'
    name = PrimaryKey(str)
    age  = Optional(int)
    height = Optional(int)
    gender = Optional(str)
    max_heart_rate = Optional(int)
    group = Optional(str)
    base_line_cal = Optional(float)
    calories = Set('Calories')
    BPM = Set('BPM')
    exercises = Set('Exercises')
    sleep = Set('Sleep')
    day = Set('Day')

class Calories(DB.Entity):
    _table_ = 'calories'
    value = Required(float)
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    
class BPM(DB.Entity):
    _table_ = 'bpm'
    vt = Required(datetime)
    value = Optional(int)
    confidence = Optional(int)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)

class Exercises(DB.Entity):
    _table_ = "exercises"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    activityName = Required(str)
    avg_BPM = Optional(float)
    calories = Optional(int)
    duration = Optional(int)
    steps = Optional(float)
    elevationGain = Optional(float)
    PrimaryKey(vt, partecipant)

class Sleep(DB.Entity):
    _table_ ="sleep"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    # attributes of the json
    duration = Optional(float)
    endTime = Optional(datetime)
    minutesAsleep = Optional(int)
    types = Optional(str)
    efficiency = Optional(int)
    PrimaryKey(partecipant, vt)

class Day(DB.Entity):
    _tabel_ = "day"
    vt = Required(datetime)
    partecipant = Required(Partecipant)
    PrimaryKey(partecipant, vt)
    n_lightly_active_minutes = Optional(int)
    n_moderately_active_minutes = Optional(int)
    f_resting_heart_rate = Optional(float)
    n_sedentary_minutes = Optional(int)
    n_very_active_minutes = Optional(int)
    n_overall_score = Optional(int)
    n_composition_score = Optional(int)
    n_revitalization_score = Optional(int)
    n_duration_score = Optional(int)
    n_deep_sleep_in_minutes = Optional(int)
    f_time_in_heart_rate_BELOW_DEFAULT_ZONE_1 = Optional(float)
    f_time_in_heart_rate_zone_1 = Optional(float)
    f_time_in_heart_rate_zone_2 = Optional(float)
    f_time_in_heart_rate_zone_3 = Optional(float)
    

DB.generate_mapping()

def extract_entity_name(entity):
    return re.split('\.|"|\'', str(entity))[-2]
    
count(r for r in eval(extract_entity_name(Calories)))

str(datetime.now())


# @db_session
# def entity_display(entity_name):
#        return f'at {str(datetime.now())[:-7]} the number of instances in {entity_name} is  {count(r for r in eval(entity_name))}'

# entity_display('Calories')

# jobs = bg.BackgroundJobManager()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])


## DASHBORD 1

In [10]:
@db_session
def pie_graph_by_partecipant(name = None):
    df = 0
    a = 0
    df  = pd.DataFrame(DB.select("""select distinct d.partecipant
              , sum(s.duration)
              , sum(d.n_sedentary_minutes)
              , sum(d.n_lightly_active_minutes)
              , sum(d.n_moderately_active_minutes)
              , sum(d.n_very_active_minutes)
from day d
join sleep s on s.partecipant = d.partecipant and date(s.vt) = d.vt
group by d.partecipant
"""), columns= ["Partecipant","Total duration sleep", "Sedentary Minutes","Lightly Active Minutes","Moderately Active Minutes","Very Active Minutes"])


    a = df.melt(id_vars=["Partecipant"],
            var_name="Type",
            value_name="Minutes").sort_values(['Partecipant']).reset_index(drop=True)

    a = a[ a["Partecipant"] == name ] if name != None else a
    fig = px.pie(a, values= 'Minutes', names='Type', color_discrete_sequence=px.colors.sequential.RdBu)

    return fig


In [11]:
def partecipant_selector():
    a = pd.DataFrame(DB.select("""select distinct d.partecipant
              , sum(s.duration)
              , sum(d.n_sedentary_minutes)
              , sum(d.n_lightly_active_minutes)
              , sum(d.n_moderately_active_minutes)
              , sum(d.n_very_active_minutes)
        from day d
            join sleep s on s.partecipant = d.partecipant
        group by d.partecipant
        """), columns= ["Partecipant","TotSleep","LightlyActiveMinutes","ModeratelyActiveMinutes","VeryActiveMinutes", "SedentaryMinutes"])
    part = list(set(a["Partecipant"]))
    part.sort()
    r = dcc.Dropdown(id='selected_part',
    options= [  { 'label': y , 'value': y } for y in part ]
    )
    return r

partecipant_selector()

Dropdown(options=[{'label': 'p01', 'value': 'p01'}, {'label': 'p02', 'value': 'p02'}, {'label': 'p03', 'value': 'p03'}, {'label': 'p04', 'value': 'p04'}, {'label': 'p05', 'value': 'p05'}, {'label': 'p07', 'value': 'p07'}, {'label': 'p09', 'value': 'p09'}, {'label': 'p11', 'value': 'p11'}], id='selected_part')

## DASHBORD2

In [12]:
@db_session
def bar_by_age_range(my_range):
        
        df3 = pd.DataFrame(DB.select("""select date(s.vt), s.partecipant, p.gender, p.age,case when d.n_deep_sleep_in_minutes is null then 0 else d.n_deep_sleep_in_minutes end AS totDeepSleep
                                        , sum(duration) - case when d.n_deep_sleep_in_minutes is null then 0 else d.n_deep_sleep_in_minutes end AS NormalSleep
                                        from sleep s
                                        join day d on d.partecipant = s.partecipant and date(s.vt) = date(d.vt)
                                        left join partecipant p on p.name = s.partecipant
                                        --where s.partecipant = 'p03'
                                        group by date(s.vt), s.partecipant, p.gender, p.age, d.n_deep_sleep_in_minutes
                                        order by date(s.vt),s.partecipant
                                """), columns= ["Data","Partecipant", "Gender", "Age","Total Deep Sleep","Total Normal Sleep",])

        
        b = df3.melt(id_vars=["Data", "Partecipant", "Gender", "Age"], 
                var_name="Sleep", 
                value_name="Minutes").sort_values(['Data', 'Partecipant']).reset_index(drop=True)
        b = b[b["Age"].between(my_range[0], my_range[1])]
        print(b)

        # fig = px.bar(b, x="Minutes", y="Partecipant", color="Sleep", title="Sleep vs Deep sleep", barmode="group")
        fig = px.bar(b, x="Data", y="Minutes", color="Partecipant", title="Sleep vs Deep sleep",pattern_shape = "Sleep", barmode="group")
        # fig.show()
        return fig


## DASHBOARD 3

In [13]:
import plotly.express as px
import calendar

@db_session
def scatter_by_month(year = None, month = None):
    data = list(select( 
        ((a.vt, a.vt.date().year, a.vt.date().month, a.activityName, a.avg_BPM, a.calories, c.name, c.age) for a in Exercises
        for c in Partecipant 
        if a.partecipant == c
        )
        ))
    print(year)
    print(month)
    data.sort(key=lambda x: [x[0]])   
    data
    df = pd.DataFrame(data, columns=['Data', 'Year', 'Month','Activity name','Bpm','Cal','Partecipant','Age'])
    df['Data'] = pd.to_datetime(df['Data'])
    df['Year'] = df['Data'].dt.year
    df['Month'] = df['Data'].dt.month

    #print("DataFrame after datetime conversion:")
   # print(df)

    if year is not None:
        df = df[df['Year'] == year]
    if month is not None:
        df = df[df['Month'] == month]
 #   print(df)

    fig = px.scatter(
        data_frame = df,
        x = "Data",
        y= "Bpm",
        size= "Cal",
        color = 'Partecipant'
    )

    fig2 = px.line(df, x='Data', y='Bpm', color='Partecipant', markers=True)
    fig2.update_layout(showlegend = False)

    fig2.for_each_trace(
        lambda trace:
        trace.update(showlegend = False)
    )

    # fig.add_trace(fig2.data[0], )
    for t in fig2.data:
        fig.add_trace(t)

    #fig.show()
    return fig

#scatter_by_month(2019,12)

@db_session
def get_available_years():
    # Query per ottenere gli anni disponibili dal database utilizzando Pony ORM
    years = list(select((y.vt.date().year) for y in Exercises))
    return years

@db_session
def get_available_months(year):
    # Query per ottenere i mesi disponibili dal database per l'anno selezionato utilizzando Pony ORM
    months = list(select(m.vt.date().month for m in Exercises if m.vt.date().year == year)) 
    #print(months)
    return months


In [14]:
layout_main = dbc.Row(
[
    dbc.Col(
        children = [
            app_title,
            app_p,
            app_items,
            app_title_dash1,
            subtitle_dash1,
            partecipant_selector(),
            dcc.Graph(id = 'chart_pie'),
            html.Div(id = 'debug1')
        ],
        **style_main_adaptive
    ),
    hr,
    dbc.Col(
        children = [
            app_title_dash2,
            subtitle_dash2,
            dcc.RangeSlider(id='my-slider',
                    marks={i: str(i) for i in range(10, 111, 5)},
                    min=10,
                    max=110,
                    value=[30,70]
            ),
            dcc.Graph(id = 'histogram'),
            html.Div(id = 'debug2')
        ],
        **style_main_adaptive
    ),
    hr,
    dbc.Col(
        children=[
            app_title_dash3,
            subtitle_dash3,
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': str(year), 'value': year} for year in get_available_years()],
            placeholder='Select a year...'
        ),
        dcc.Dropdown(
            id='month-dropdown',
            options=[],
            placeholder='Select a month...'
        ),
        html.Div(id='selected-info'),  # Div to display the selected year and month
        dcc.Graph(id='scatter-plot')
    ],
    **style_main_adaptive
    )
]
)
app.layout = layout_main
app.layout


Row([Col(children=[H1(children='Welcome to the dashboard page of HIS exam', style={'text-align': 'center'}), P('We choose to implement an html page written in python with bootstrap that permits to visualize pmdata dataset stored previously on the database with ORM. We implement 3 different types of dashboard, each with a callback function to filter data visualized:'), Ul([Li(['Pie graph']), Li(['Bar graph']), Li(['Scatter graph'])]), H2('PIE GRAPH - Dashboard 1'), H4(children='Visualize the minutes in different activity for partecipants', style={'font-style': 'italic', 'color': 'grey'}), Dropdown(options=[{'label': 'p01', 'value': 'p01'}, {'label': 'p02', 'value': 'p02'}, {'label': 'p03', 'value': 'p03'}, {'label': 'p04', 'value': 'p04'}, {'label': 'p05', 'value': 'p05'}, {'label': 'p07', 'value': 'p07'}, {'label': 'p09', 'value': 'p09'}, {'label': 'p11', 'value': 'p11'}], id='selected_part'), Graph(id='chart_pie'), Div(id='debug1')], lg={'size': 8, 'offset': 2}, md={'size': 12, 'offse

In [15]:
@app.callback(
    Output('chart_pie', 'figure'),
    Input('selected_part', 'value')
)
def pie_by_partecipant(value):
    if value != None:
        return pie_graph_by_partecipant(value)
    else:
        return pie_graph_by_partecipant()
    
@app.callback(
    Output('histogram', 'figure'),
    Input('my-slider', 'value')
)
def bar_age_range(slider_range):
    if slider_range != None:
        return bar_by_age_range(slider_range)
    else:
        return bar_by_age_range()

In [16]:
import plotly.graph_objects as go


# Define a callback function to update the dropdown options based on the data
@app.callback(
    Output('month-dropdown', 'options'),
    [Input('year-dropdown', 'value')]
)
def update_month_dropdown(year):
    if year is None:
        month_options = []
    else:
        months = get_available_months(year)
        month_options = [{'label': calendar.month_name[month], 'value': month} for month in months]
    return month_options


@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('year-dropdown', 'value'),
     Input('month-dropdown', 'value')]
)
def update_graph(selected_year, selected_month):
    if selected_year is not None and selected_month is not None:
        #month_number = list(calendar.month_name).index(selected_month)
        selected_month = calendar.month_name[selected_month]
        month_number = list(calendar.month_name).index(selected_month)
        fig = scatter_by_month(selected_year, month_number)
    else:
        fig = go.Figure()
        fig = scatter_by_month()
        # In caso di mancata selezione di un anno o un mese, puoi restituire un grafico vuoto o un messaggio di avviso
        fig.update_layout(title='Please select a year and month')
    return fig


In [ ]:
jobs.new("app.run_server( host='127.0.0.1', port='8051')")

Dash is running on http://127.0.0.1:8051/



INFO:dash.dash:Dash is running on http://127.0.0.1:8051/



<BackgroundJob #0: app.run_server( host='127.0.0.1', port='8051')>

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8051
INFO:werkzeug:Press CTRL+C to quit


...

INFO:werkzeug:127.0.0.1 - - [20/Jul/2023 14:30:49] "POST /_dash-update-component HTTP/1.1" 200 -


2020
1


INFO:werkzeug:127.0.0.1 - - [20/Jul/2023 14:30:52] "POST /_dash-update-component HTTP/1.1" 200 -


2020
2


INFO:werkzeug:127.0.0.1 - - [20/Jul/2023 14:30:55] "POST /_dash-update-component HTTP/1.1" 200 -
